# Urls from Mediacloud to news articles:

**Tasks:**
- Retrieve html data from urls
- Use NewsPlease to get articles from html data

In [24]:
from dotenv import load_dotenv
import os, mediacloud.api
import datetime
import json
import pandas as pd 
from IPython.display import JSON
import mediacloud.tags
import csv
import requests
import time
import newsplease
from tqdm import tqdm
from newsplease import NewsPlease
from urllib.parse import urlparse

ModuleNotFoundError: No module named 'newsplease'

In [14]:
import pandas as pd 
from tqdm import tqdm
import json

In [4]:
# First, we load the url data retrieved from Mediacloud
mc_urls = pd.read_csv('C:/Users/2146806A/OneDrive - University of Glasgow/University of Glasgow/Amsterdam Visit/March_analysis/newdata/newdata.csv')
mc_urls.head() 

,archived_url,article_url,id,language,media_name,media_url,publish_date,title,url
0,https://web.archive.org/web/20220611074034/htt...,https://wayback-api.archive.org/colsearch/v1/m...,2.022060e+13,en,breitbart.com,http://breitbart.com,10/06/2022 00:00,"FNC's Carlson: January 6 Hearings 'Insulting,'...",https://www.breitbart.com/clips/2022/06/10/fnc...
1,https://web.archive.org/web/20220614071134/htt...,https://wayback-api.archive.org/colsearch/v1/m...,2.022060e+13,en,breitbart.com,http://breitbart.com,13/06/2022 00:00,"Poll: Plurality Say Biden, MLB Should Apologiz...",https://www.breitbart.com/politics/2022/06/13/...
2,https://web.archive.org/web/20220712173353/htt...,https://wayback-api.archive.org/colsearch/v1/m...,2.022070e+13,en,breitbart.com,http://breitbart.com,11/07/2022 00:00,Biden’s Medal of Freedom Recipient Has Worked ...,https://www.breitbart.com/politics/2022/07/11/...
3,https://web.archive.org/web/20220818074856/htt...,https://wayback-api.archive.org/colsearch/v1/m...,2.022080e+13,en,breitbart.com,http://breitbart.com,17/08/2022 00:00,Kent: Trump Raid Is GOP Establishment's 'Road ...,https://www.breitbart.com/politics/2022/08/17/...
4,https://web.archive.org/web/20220819083054/htt...,https://wayback-api.archive.org/colsearch/v1/m...,2.022080e+13,en,breitbart.com,http://breitbart.com,18/08/2022 00:00,9 Times Sen. Lisa Murkowski Voted with Democrats,https://www.breitbart.com/politics/2022/08/18/...


In [5]:
pd.DataFrame(mc_urls['media_url'].value_counts())  #counting urls per news media source

,media_url
http://nytimes.com,61
http://foxnews.com,45
http://dailycaller.com,24
http://washingtontimes.com,23
http://cnn.com,22
http://breitbart.com,18
http://nbcnews.com,11
http://nypost.com,11
http://freebeacon.com,3
http://usatoday.com,3


In [ ]:
# Find source from url link
def urltosource(link):
    source = urlparse(link).netloc
    return source

source = urltosource(data['url'][1]) #checking if it works
source

In [19]:
# Create a source column from urls
mc_urls['source'] = data['url'].apply(urltosource)

In [6]:
mc_urls.head() 

,archived_url,article_url,id,language,media_name,media_url,publish_date,title,url
0,https://web.archive.org/web/20220611074034/htt...,https://wayback-api.archive.org/colsearch/v1/m...,2.022060e+13,en,breitbart.com,http://breitbart.com,10/06/2022 00:00,"FNC's Carlson: January 6 Hearings 'Insulting,'...",https://www.breitbart.com/clips/2022/06/10/fnc...
1,https://web.archive.org/web/20220614071134/htt...,https://wayback-api.archive.org/colsearch/v1/m...,2.022060e+13,en,breitbart.com,http://breitbart.com,13/06/2022 00:00,"Poll: Plurality Say Biden, MLB Should Apologiz...",https://www.breitbart.com/politics/2022/06/13/...
2,https://web.archive.org/web/20220712173353/htt...,https://wayback-api.archive.org/colsearch/v1/m...,2.022070e+13,en,breitbart.com,http://breitbart.com,11/07/2022 00:00,Biden’s Medal of Freedom Recipient Has Worked ...,https://www.breitbart.com/politics/2022/07/11/...
3,https://web.archive.org/web/20220818074856/htt...,https://wayback-api.archive.org/colsearch/v1/m...,2.022080e+13,en,breitbart.com,http://breitbart.com,17/08/2022 00:00,Kent: Trump Raid Is GOP Establishment's 'Road ...,https://www.breitbart.com/politics/2022/08/17/...
4,https://web.archive.org/web/20220819083054/htt...,https://wayback-api.archive.org/colsearch/v1/m...,2.022080e+13,en,breitbart.com,http://breitbart.com,18/08/2022 00:00,9 Times Sen. Lisa Murkowski Voted with Democrats,https://www.breitbart.com/politics/2022/08/18/...


In [7]:
print(mc_urls.shape[0])

221


In [8]:
url_counts=pd.DataFrame(mc_urls['media_name'].value_counts())
url_counts

,media_name
nytimes.com,61
foxnews.com,45
dailycaller.com,24
washingtontimes.com,23
cnn.com,22
breitbart.com,18
nbcnews.com,11
nypost.com,11
freebeacon.com,3
usatoday.com,3


In [9]:
mc_urls.head()

,archived_url,article_url,id,language,media_name,media_url,publish_date,title,url
0,https://web.archive.org/web/20220611074034/htt...,https://wayback-api.archive.org/colsearch/v1/m...,2.022060e+13,en,breitbart.com,http://breitbart.com,10/06/2022 00:00,"FNC's Carlson: January 6 Hearings 'Insulting,'...",https://www.breitbart.com/clips/2022/06/10/fnc...
1,https://web.archive.org/web/20220614071134/htt...,https://wayback-api.archive.org/colsearch/v1/m...,2.022060e+13,en,breitbart.com,http://breitbart.com,13/06/2022 00:00,"Poll: Plurality Say Biden, MLB Should Apologiz...",https://www.breitbart.com/politics/2022/06/13/...
2,https://web.archive.org/web/20220712173353/htt...,https://wayback-api.archive.org/colsearch/v1/m...,2.022070e+13,en,breitbart.com,http://breitbart.com,11/07/2022 00:00,Biden’s Medal of Freedom Recipient Has Worked ...,https://www.breitbart.com/politics/2022/07/11/...
3,https://web.archive.org/web/20220818074856/htt...,https://wayback-api.archive.org/colsearch/v1/m...,2.022080e+13,en,breitbart.com,http://breitbart.com,17/08/2022 00:00,Kent: Trump Raid Is GOP Establishment's 'Road ...,https://www.breitbart.com/politics/2022/08/17/...
4,https://web.archive.org/web/20220819083054/htt...,https://wayback-api.archive.org/colsearch/v1/m...,2.022080e+13,en,breitbart.com,http://breitbart.com,18/08/2022 00:00,9 Times Sen. Lisa Murkowski Voted with Democrats,https://www.breitbart.com/politics/2022/08/18/...


In [18]:
# We create a list of urls from pandas dataframe
urls = mc_urls['url'].to_list()

In [19]:
# retrieve html pages from list of urls 
allpages = {}
for url in tqdm(urls):
    try:
        r = requests.get(url)
        if r.status_code==200:
            allpages[url] = r.text #only read the html text if page exists
        else:
            print(f'Downloading {url} returned status code {r.status_code}')
    except:
        print(f'some error occurred when downloading {url}')
with open('TMP.json',  mode='w') as f:
    json.dump(allpages, f)

100%|██████████| 221/221 [00:00<00:00, 219967.06it/s]

some error occurred when downloading https://www.breitbart.com/clips/2022/06/10/fncs-carlson-january-6-hearings-insulting-deranged-they-are-lying/
some error occurred when downloading https://www.breitbart.com/politics/2022/06/13/poll-nearly-half-of-voters-say-biden-mlb-should-apologize-to-georgia-for-attacking-election-integrity-law/
some error occurred when downloading https://www.breitbart.com/politics/2022/07/11/one-of-bidens-medal-of-freedom-recipients-has-a-history-of-working-with-controversial-far-left-groups/
some error occurred when downloading https://www.breitbart.com/politics/2022/08/17/exclusive-joe-kent-trump-raid-is-gop-establishments-road-to-damascus-moment/
some error occurred when downloading https://www.breitbart.com/politics/2022/08/18/9-times-lisa-murkowski-voted-democrats/
some error occurred when downloading https://www.breitbart.com/politics/2022/08/28/exclusive-republicans-escalate-push-poll-worker-transparency-clark-county-nevada/
some error occurred when down

In [20]:
len(allpages) # check how many pages were retrieved


0

In [9]:
allpages_parsed = {}
for k, v in tqdm(allpages.items()):
    allpages_parsed[k] = NewsPlease.from_html(v) # use NewsPlease here to download the stories from the html info retrieved

100%|██████████| 161/161 [00:58<00:00,  2.74it/s]


In [10]:
with open("output_newsplease.json", mode = 'w') as f: #write json file with media data
    for k, v in allpages_parsed.items():
        mydict = v.get_serializable_dict()
        mydict['url'] = k
        f.write(json.dumps(mydict))
        f.write("\n")

In [11]:
data = pd.read_json('output_newsplease.json', lines=True) # convert to dataframe
data.head()

,authors,date_download,date_modify,date_publish,description,filename,image_url,language,localpath,maintext,source_domain,text,title,title_page,title_rss,url
0,[Jeff Poor],None,None,2022-06-10 13:50:52,"Thursday, Fox News Channel's Tucker Carlson ha...",.json,https://media.breitbart.com/media/2021/08/Carl...,en,NaN,"Thursday, Fox News Channel’s Tucker Carlson ha...",NaN,NaN,"January 6 Hearings 'Insulting,' 'Deranged' -- ...",NaN,NaN,https://www.breitbart.com/clips/2022/06/10/fnc...
1,[Hannah Bleau],None,None,2022-06-13 16:14:46,Nearly a majority of voters believe President ...,.json,https://media.breitbart.com/media/2021/04/Carm...,en,NaN,Nearly a majority of voters believe President ...,NaN,NaN,"Plurality Say Biden, MLB Should Apologize to G...",NaN,NaN,https://www.breitbart.com/politics/2022/06/13/...
2,[Jacob Bliss],None,None,2022-07-11 21:37:32,"Raul Yzaguirre, one of President Joe Biden’s M...",.json,https://media.breitbart.com/media/2022/07/la_r...,en,NaN,"Raul Yzaguirre, one of President Joe Biden’s M...",NaN,NaN,Biden’s Medal of Freedom Recipient Has Worked ...,NaN,NaN,https://www.breitbart.com/politics/2022/07/11/...
3,[Kristina Wong],None,None,2022-08-18 01:17:44,Kent called for Republicans to unify after the...,.json,https://media.breitbart.com/media/2021/07/Joe-...,en,NaN,"Green Beret veteran Joe Kent, fresh from his R...",NaN,NaN,Trump Raid Is GOP Establishment's 'Road to Dam...,NaN,NaN,https://www.breitbart.com/politics/2022/08/17/...
4,[Wendell Husebø],None,None,2022-08-18 20:28:42,Sen. Lisa Murkowski heads into the midterm ele...,.json,https://media.breitbart.com/media/2021/11/Joe-...,en,NaN,Sen. Lisa Murkowski (R-AK) heads into the Nove...,NaN,NaN,9 Times Sen. Lisa Murkowski Voted with Democrats,NaN,NaN,https://www.breitbart.com/politics/2022/08/18/...


In [39]:
# Create a source column from urls
data['source'] = data['url'].apply(urltosource)
data.head()
len(data)

161

In [15]:
df_dates = data.groupby('source').agg({'date_publish': ['min', 'max']}).reset_index()
df_dates.columns = df_dates.columns.to_flat_index()
df_dates = df_dates.rename(columns={df_dates.columns[0]: "media_name",
                                      df_dates.columns[1]: "min_retrieved_date",
                                      df_dates.columns[2]: "max_retrieved_date"})

df_dates

,media_name,min_retrieved_date,max_retrieved_date
0,dailycaller.com,2022-04-22 17:40:50,2022-12-07 12:57:02
1,freebeacon.com,2022-05-23 22:36:26,2022-09-12 19:15:50
2,nypost.com,2022-05-08 01:23:47,2022-12-07 22:57:42
3,radio.foxnews.com,2022-11-09 10:25:45,2022-11-09 10:25:45
4,www.breitbart.com,2022-06-10 13:50:52,2022-12-06 20:04:40
5,www.cnn.com,2022-04-05 22:23:06,2022-12-18 12:18:30
6,www.foxnews.com,2022-04-14 13:58:14,2023-01-09 15:53:35
7,www.nbcnews.com,2022-05-16 16:23:22,2022-12-22 00:00:00
8,www.nytimes.com,2022-09-20 15:08:11,2022-12-19 19:28:53
9,www.usatoday.com,2022-09-06 09:01:25,2022-11-03 02:40:53


In [44]:
# create dataframe of urls per news media
url_counts = pd.DataFrame(mc_urls['source'].value_counts())
url_counts

,source
www.foxnews.com,44
dailycaller.com,23
www.washingtontimes.com,23
www.cnn.com,22
www.breitbart.com,18
www.nbcnews.com,11
nypost.com,11
freebeacon.com,3
www.usatoday.com,3
www.nytimes.com,2


In [45]:
sources_counts = pd.DataFrame(data['source'].value_counts())
sources_counts

,source
www.foxnews.com,44
dailycaller.com,23
www.washingtontimes.com,23
www.cnn.com,22
www.breitbart.com,18
www.nbcnews.com,11
nypost.com,11
freebeacon.com,3
www.usatoday.com,3
www.nytimes.com,2


In [29]:
# merge the two dataframes on the media source column
merged = pd.merge(url_counts, source_counts, left_index=True, right_index=True)
merged.head()

,source_x,source_y
www.foxnews.com,44,44
dailycaller.com,23,23
www.washingtontimes.com,23,23
www.cnn.com,22,22
www.breitbart.com,18,18


In [ ]:
# calculate the percentage of stories retrieved per media source
merged['percentage'] = (merged['source'] / merged['source']) * 100

# print the new dataframe
print(merged)

In [24]:
# save data to csv file
data.to_csv('C:/Users/analo/OneDrive - University of Glasgow/University of Glasgow/Amsterdam Visit/March_analysis/newdata/newsplease_corpus.csv', index = False)